In [1]:
import findspark
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd
import numpy as np  
    
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

import warnings
warnings.filterwarnings('ignore')

findspark.init()
spark = SparkSession \
    .builder \
    .appName("anova") \
    .config('spark.sql.session.timeZone', 'Asia/Shanghai') \
    .master("local[*]") \
    .getOrCreate()

In [11]:
class AnovaAnalysis:
    """
    单/多因素方差分析  
    :param table_name: 数据dataframe
    :param factor: 单/多个因素(类型变量),应为列表形式
    :param value: 因变量(连续型或数值型变量)
    :param interaction: 有无交互作用，默认有
    :return: 方差分析表
    """
    def __init__(self, table_name: spark_dataframe_type, factor:list[str], value: str, interaction = True):
        self.table_name = table_name
        self.factor = factor
        self.value = value
        self.interaction = interaction
    
    
    # 将factor变为C(a) + C(b) + ...的形式；如果有交互作用，还需要C(a)：C(b) 
    @staticmethod
    def convert_factor(factor, interaction):
        tran_factor = [f'C({x})' for x in factor]
        add_tran_factor = "+".join(tran_factor)
        if interaction:
            inter_factor = ":".join(tran_factor)
            return "+".join([add_tran_factor, inter_factor])
        else:
            return add_tran_factor
    
    
    # 主函数：利用statsmodel中方差分析的函数，返回方差分析表
    def run(self):
        expression = self.convert_factor(self.factor, self.interaction)
        model = ols(f"{self.value} ~ {expression}", self.table_name).fit()
        anova_results = anova_lm(model)
        return anova_results

In [12]:
d = np.array([
    [58.2, 52.6, 56.2, 41.2, 65.3, 60.8],
    [49.1, 42.8, 54.1, 50.5, 51.6, 48.4],
    [60.1, 58.3, 70.9, 73.2, 39.2, 40.7],
    [75.8, 71.5, 58.2, 51.0, 48.7, 41.4]])
df = pd.DataFrame(d)
df.index = pd.Index(['A1', 'A2', 'A3', 'A4'], name='燃料')
df.columns = pd.Index(['B1', 'B1', 'B2', 'B2', 'B3', 'B3'], name='推进器')
df1 = df.stack().reset_index().rename(columns={0: '射程'})
df_spark = spark.createDataFrame(df1)
df_spark.show()

+----+------+----+
|燃料|推进器|射程|
+----+------+----+
|  A1|    B1|58.2|
|  A1|    B1|52.6|
|  A1|    B2|56.2|
|  A1|    B2|41.2|
|  A1|    B3|65.3|
|  A1|    B3|60.8|
|  A2|    B1|49.1|
|  A2|    B1|42.8|
|  A2|    B2|54.1|
|  A2|    B2|50.5|
|  A2|    B3|51.6|
|  A2|    B3|48.4|
|  A3|    B1|60.1|
|  A3|    B1|58.3|
|  A3|    B2|70.9|
|  A3|    B2|73.2|
|  A3|    B3|39.2|
|  A3|    B3|40.7|
|  A4|    B1|75.8|
|  A4|    B1|71.5|
+----+------+----+
only showing top 20 rows



In [13]:
d = np.array([
    [58.2, 52.6, 56.2, 41.2, 65.3, 60.8],
    [49.1, 42.8, 54.1, 50.5, 51.6, 48.4],
    [60.1, 58.3, 70.9, 73.2, 39.2, 40.7],
    [75.8, 71.5, 58.2, 51.0, 48.7,41.4]])
df = pd.DataFrame(d)
df.index=pd.Index(['A1','A2','A3','A4'],name='燃料')
df.columns=pd.Index(['B1','B1','B2','B2','B3','B3'],name='推进器')
df1 = df.stack().reset_index().rename(columns={0: '射程'})

mwa = AnovaAnalysis(df1, ['燃料', '推进器'], '射程', interaction=True)
mwa.run()

,df,sum_sq,mean_sq,F,PR(>F)
C(燃料),3.0,261.675000,87.225000,4.417388,0.025969
C(推进器),2.0,370.980833,185.490417,9.393902,0.003506
C(燃料):C(推进器),6.0,1768.692500,294.782083,14.928825,0.000062
Residual,12.0,236.950000,19.745833,NaN,NaN


In [14]:
print(type(df1 ))
model = ols('射程 ~ C(燃料)+C(推进器)+C(燃料):C(推进器)', df1).fit()
anova_results = anova_lm(model)
anova_results

<class 'pandas.core.frame.DataFrame'>


,df,sum_sq,mean_sq,F,PR(>F)
C(燃料),3.0,261.675000,87.225000,4.417388,0.025969
C(推进器),2.0,370.980833,185.490417,9.393902,0.003506
C(燃料):C(推进器),6.0,1768.692500,294.782083,14.928825,0.000062
Residual,12.0,236.950000,19.745833,NaN,NaN


In [15]:
import findspark
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd
import numpy as np  
    
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

import warnings
warnings.filterwarnings('ignore')

findspark.init()
spark = SparkSession \
    .builder \
    .appName("anova") \
    .config('spark.sql.session.timeZone', 'Asia/Shanghai') \
    .master("local[*]") \
    .getOrCreate()

In [16]:
df_spark = spark.createDataFrame(df1)
print(type(df_spark ))

df_spark1 = df_spark.toPandas()
print(type(df_spark1 ))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [17]:
model = ols('射程 ~ C(燃料)+C(推进器)+C(燃料):C(推进器)', df_spark1).fit()
anova_results = anova_lm(model)
anova_results.reset_index(names='factor')

,factor,df,sum_sq,mean_sq,F,PR(>F)
0,C(燃料),3.0,261.675000,87.225000,4.417388,0.025969
1,C(推进器),2.0,370.980833,185.490417,9.393902,0.003506
2,C(燃料):C(推进器),6.0,1768.692500,294.782083,14.928825,0.000062
3,Residual,12.0,236.950000,19.745833,NaN,NaN


In [18]:
import pyspark.pandas as ps
type(ps.from_pandas(anova_results).to_spark())

pyspark.sql.dataframe.DataFrame

检查字段名是否含有特殊字符

In [134]:
import pandas as pd
  
Data = {'Name#': ['Mukul', 'Rohan', 'Mayank',
                  'Shubham', 'Aakash'],    
        'Locat    ion': ['Saharanpur', 'Meerut', 'Agra',
                     'Saharanpur', 'Meerut'],
        'Pay!': [25000, 30000, 35000, 40000, 45000]}
  
df = pd.DataFrame(Data)
original_columns = pd.Series(df.columns)
df

,Name#,Locat ion,Pay!
0,Mukul,Saharanpur,25000
1,Rohan,Meerut,30000
2,Mayank,Agra,35000
3,Shubham,Saharanpur,40000
4,Aakash,Meerut,45000


In [135]:
# remove special character
df.columns = df.columns.str.replace('[#,@,&,!,., ,?]', '', regex=True)
df

,Name,Location,Pay
0,Mukul,Saharanpur,25000
1,Rohan,Meerut,30000
2,Mayank,Agra,35000
3,Shubham,Saharanpur,40000
4,Aakash,Meerut,45000


In [136]:
value = df.columns[2]
factor = df.columns[0]  # 'Name'

model = ols(f'{value} ~ {factor}', df).fit()
anova_results = anova_lm(model)
anova_results = anova_results.reset_index()
anova_results

,index,df,sum_sq,mean_sq,F,PR(>F)
0,Name,4.0,2.500000e+08,62500000.0,0.0,NaN
1,Residual,0.0,1.813180e-21,inf,NaN,NaN


In [137]:
anova_results_loc = np.where(anova_results['index'].str.contains(factor))
original_columns_loc = np.where(original_columns.str.contains(factor)) 
anova_results['index'].iloc[anova_results_loc] = original_columns.iloc[original_columns_loc].values

In [138]:
anova_results

,index,df,sum_sq,mean_sq,F,PR(>F)
0,Name#,4.0,2.500000e+08,62500000.0,0.0,NaN
1,Residual,0.0,1.813180e-21,inf,NaN,NaN
